In [1]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.losses import mse, binary_crossentropy
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Concatenate, Conv1D, Reshape, Flatten, MaxPool1D, GlobalMaxPool1D, GlobalAvgPool1D, Multiply, Dot, Input, Attention, BatchNormalization, Dropout, Lambda, Input, Dense, Concatenate
from tensorflow.keras import backend as K
from tensorflow.keras import metrics
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
#tf.keras.backend.clear_session()
# tf.config.experimental_run_functions_eagerly(True)

In [3]:
import pandas as pd
import numpy as np
# 폰트 설정 방법 1
import matplotlib.pyplot as plt
# matplotlib 폰트설정
# plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rc('font', family='NanumGothic') # For Windows
print(plt.rcParams['font.family'])
pd.set_option('display.max_rows', 100)

ans = pd.read_csv("answer_sheet.csv")
stocks = pd.read_csv('stocks.csv')
trade = pd.read_csv('trade_train.csv')

num2name = dict((stocks[['종목번호','종목명']].drop_duplicates()).values)

['NanumGothic']


In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le2 = LabelEncoder()

col = '종목번호'
le.fit(stocks[col].append(trade[col]))
stocks[col] = le.transform(stocks[col].fillna('NaN'))
#stocks[col] = stocks[col].astype('category')
trade[col] = le.transform(trade[col].fillna('NaN'))
#trade[col] = trade[col].astype('category')
            
col = '그룹번호'
le2.fit(trade[col])
trade[col] = le2.transform(trade[col].fillna('NaN'))
#trade[col] = trade[col].astype('category')

# 전고점, 전저점 변수 생성
from tqdm import tqdm
stocks['전고점돌파'] = np.nan
stocks['전저점돌파'] = np.nan
for i in tqdm(sorted(set(stocks['종목번호']))):
    global_max = 0
    global_min = 999999999999
    for j in sorted(set(stocks.loc[stocks['종목번호']==i, '기준일자'])):
        idx = (stocks['종목번호']==i) & (stocks['기준일자']==j)
        if stocks.loc[idx, '종목고가'].values[0] >= global_max:
            global_max = stocks.loc[idx, '종목고가'].values[0]
            stocks.loc[idx, '전고점돌파'] = 1
        if stocks.loc[idx, '종목저가'].values[0] <= global_min:
            global_min = stocks.loc[idx, '종목저가'].values[0]
            stocks.loc[idx, '전저점돌파'] = 1

stocks[['전고점돌파','전저점돌파']] = stocks[['전고점돌파','전저점돌파']].fillna(0)



# trade 파생변수
trade['매수고객수_비율'] = trade['매수고객수']/trade['그룹내고객수']
trade['매도고객수_비율'] = trade['매도고객수']/trade['그룹내고객수']
trade['평균매수단가'] = trade['평균매수수량']*trade['매수가격_중앙값']*trade['매수고객수']
trade['평균매도단가'] = trade['평균매도수량']*trade['매도가격_중앙값']*trade['매도고객수']

100%|██████████| 1087/1087 [07:55<00:00,  2.29it/s]


In [5]:
stocks['기준년월'] = stocks['기준일자']//100

In [6]:
high_low_sum = stocks.groupby(['종목번호','기준년월'])[['전고점돌파','전저점돌파']].sum().reset_index()
stocks_max = stocks.loc[(stocks.groupby(["종목번호","기준년월"])["거래량"].idxmax()).dropna()]
stocks_max = pd.merge(stocks_max, high_low_sum, how='left', on=['종목번호','기준년월'], suffixes=['', '_합계'])

stocks_max['기준년월'] = stocks_max['기준년월'] - 1
stocks_max['기준년월'] = stocks_max['기준년월'].replace(202000, 201912)

trade = pd.merge(trade, stocks_max, how='left', on=['종목번호','기준년월'])
trade = trade.fillna(0)

In [7]:
# 주가 스케일링
from tqdm import tqdm
target = ['종목시가','종목고가','종목저가','종목종가', '거래량','거래금액_만원단위']
for i, j in tqdm(stocks[['기준년월','종목번호']].drop_duplicates().values):
    stocks.loc[(stocks['기준년월']==i) & (stocks['종목번호']==j), target] =  stocks.loc[(stocks['기준년월']==i) & (stocks['종목번호']==j), target]/stocks.loc[(stocks['기준년월']==i) & (stocks['종목번호']==j), target].values[0]

100%|██████████| 13912/13912 [03:27<00:00, 67.09it/s]


In [8]:
# top3 추가, 
# 월별 그룹별 매수, 매도 고객수 스케일링
#trade = trade.fillna(0)
from tqdm import tqdm
#for i in sorted(list(trade['그룹번호'])):
trade['top3'] = 0
top = 3
#i = sorted(list(trade['그룹번호']))[0]
for m in tqdm(sorted(set(trade['기준년월']))):
    for i in sorted(set(trade['그룹번호'])):
        idx = trade.loc[(trade['그룹번호']==i) & (trade['기준년월']==m),'매수고객수'].sort_values().index[-top:]
        trade.loc[idx,'top3'] = 1
        trade.loc[(trade['그룹번호']==i) & (trade['기준년월']==m),'매수고객수']/=trade.loc[(trade['그룹번호']==i) & (trade['기준년월']==m),'매수고객수'].max()
        trade.loc[(trade['그룹번호']==i) & (trade['기준년월']==m),'매도고객수']/=trade.loc[(trade['그룹번호']==i) & (trade['기준년월']==m),'매도고객수'].max()

100%|██████████| 12/12 [00:04<00:00,  2.77it/s]


In [9]:
tmp = trade.copy()
tmp['기준년월'] = tmp['기준년월'] + 1
tmp['기준년월'] = tmp['기준년월'].replace(201913, 202001)
trade = pd.merge(trade, tmp[['매수고객수','기준년월','종목번호','그룹번호']], how='left', on=['기준년월','종목번호','그룹번호'], 
        suffixes=['_정답', '_전월'])
trade = pd.merge(trade, tmp[['기준년월','종목번호','그룹번호','매수가격_중앙값','매도가격_중앙값']], how='left', on=['기준년월','종목번호','그룹번호'], 
        suffixes=['', '_전월'])

trade = trade.fillna(0)

In [10]:
tmp = trade.copy()
tmp['기준년월'] = tmp['기준년월'] + 1
tmp['기준년월'] = tmp['기준년월'].replace(201913, 202001)
trade = pd.merge(trade, tmp[['매수고객수_전월','기준년월','종목번호','그룹번호']], how='left', on=['기준년월','종목번호','그룹번호'], 
        suffixes=['', '_2전월'])

trade = trade.fillna(0)

In [11]:
col = '매수가격_중앙값'
trade[col] = trade[col].replace(-1, 0)
trade['매수가격_종목시가_비율'] = trade[col]/trade['종목시가']
trade['매수가격_종목고가_비율'] = trade[col]/trade['종목고가']
trade['매수가격_종목저가_비율'] = trade[col]/trade['종목저가']
trade['매수가격_종목종가_비율'] = trade[col]/trade['종목종가']

col = '매도가격_중앙값'
trade[col] = trade[col].replace(-1, 0)
trade['매도가격_종목시가_비율'] = trade[col]/trade['종목시가']
trade['매도가격_종목고가_비율'] = trade[col]/trade['종목고가']
trade['매도가격_종목저가_비율'] = trade[col]/trade['종목저가']
trade['매도가격_종목종가_비율'] = trade[col]/trade['종목종가']

In [12]:
trade = trade.replace([np.inf, -np.inf], np.nan).fillna(0)

# train

In [13]:
from tqdm import tqdm
month = 202003
#month = 201902
stock_info = []
trade_info = []
y_list = []
y_list_top = []

for i, j in tqdm(trade[trade['기준년월']>=month][['기준년월','종목번호']].drop_duplicates().values):
#         stock_info.append(stocks.loc[(stocks['기준년월']==i) & (stocks['종목명']==j), ['종목시가','종목고가','종목저가','종목종가','거래량'
#                                                               ,'거래금액_만원단위','전고점돌파','전저점돌파']].values)
        
        # if j not in set(stocks.loc[stocks['20년7월TOP3대상여부']=='Y','종목번호']):
        #     continue

    stock_seq = stocks.loc[(stocks['기준년월']==i) & (stocks['종목번호']==j), ['종목시가','종목고가','종목저가','종목종가','거래량'
                                                        ,'거래금액_만원단위','전고점돌파','전저점돌파']].values
    if len(stock_seq) <19 :
        continue
    elif len(stock_seq) == 19 :
        stock_seq = np.vstack((stock_seq, np.zeros((1,stock_seq.shape[1]))))
    
    # trade_seq = trade.loc[(trade['기준년월']==i) & (trade['종목번호']==j), ['그룹번호','매수고객수_전월','매수고객수_전월_2전월','top3']].values
    # y_list.append(trade.loc[(trade['기준년월']==i) & (trade['종목번호']==j), '매수고객수_정답'].values)

    trade_seq = trade.loc[(trade['기준년월']==i) & (trade['종목번호']==j), ['기준년월','종목번호','그룹번호','매수고객수_전월', '매수고객수_정답','top3']]
    trade_seq2 = trade.loc[(trade['기준년월']==i-1) & (trade['종목번호']==j), ['기준년월','종목번호','그룹번호'] + 
                                    ['전고점돌파_합계', '전저점돌파_합계','매수가격_종목시가_비율',
'매수가격_종목고가_비율', '매수가격_종목저가_비율', '매수가격_종목종가_비율', '매도가격_종목시가_비율',
'매도가격_종목고가_비율', '매도가격_종목저가_비율', '매도가격_종목종가_비율']].copy()
                                
    trade_seq2['기준년월'] += 1

    trade_seq = pd.merge(trade_seq, trade_seq2, on=['기준년월','종목번호','그룹번호'], how='left')

    y_list.append(trade_seq.loc[(trade_seq['기준년월']==i) & (trade_seq['종목번호']==j), '매수고객수_정답'].values)
    y_list_top.append(trade_seq.loc[(trade_seq['기준년월']==i) & (trade_seq['종목번호']==j), 'top3'].values)

    trade_seq = trade_seq.drop(['기준년월','종목번호','매수고객수_정답','top3'], axis=1).fillna(0).values


    trade_info.append(trade_seq)
    for _ in range(len(trade_seq)):
        stock_info.append(stock_seq[:20])
            
            #y_list.append(trade.loc[(trade['기준년월']==i) & (trade['종목번호']==j), 'top3'].values)

            
stock_info = np.array(stock_info)
trade_info = np.vstack(trade_info)
y_list = (np.hstack(y_list)).reshape(-1,1)
            


100%|██████████| 2536/2536 [00:30<00:00, 83.96it/s]


In [14]:
y_list_top = np.hstack(y_list_top).reshape(-1,1)

In [15]:
y_list_top = np.hstack((y_list_top==0, y_list_top==1)).astype(int)

In [16]:
y_list_top = np.hstack((y_list_top==0, y_list_top==1)).astype(int)

# test set

In [17]:
test_stock_info = []
test_trade_info = []
item_num = []
i = 202007
for j in tqdm(sorted(set(stocks.loc[stocks['20년7월TOP3대상여부']=='Y','종목번호']))):
    stock_seq = stocks.loc[(stocks['기준년월']==i) & (stocks['종목번호']==j), ['종목번호','종목시가','종목고가','종목저가','종목종가','거래량'
                                                            ,'거래금액_만원단위','전고점돌파','전저점돌파']].values
    if len(stock_seq) <19 :
            continue
    if len(stock_seq) == 19 :
        stock_seq = np.vstack((stock_seq, np.zeros((1,stock_seq.shape[1]))))
    
    # trade_seq = trade.loc[(trade['기준년월']==i-1) & (trade['종목번호']==j), ['그룹번호','매수고객수_정답','매수고객수_전월']].values
    # trade_seq = np.hstack((trade_seq, np.ones((len(trade_seq),1))))
    trade_seq = trade.loc[(trade['기준년월']==i-1) & (trade['종목번호']==j), ['그룹번호','매수고객수_정답'] +
    ['전고점돌파_합계', '전저점돌파_합계','매수가격_종목시가_비율',
                   '매수가격_종목고가_비율', '매수가격_종목저가_비율', '매수가격_종목종가_비율', '매도가격_종목시가_비율',
                          '매도가격_종목고가_비율', '매도가격_종목저가_비율', '매도가격_종목종가_비율']].copy().values
                                    
        
                             
    test_trade_info.append(trade_seq)
    for _ in range(len(trade_seq)):
        test_stock_info.append(stock_seq[:20])
            
test_stock_info = np.array(test_stock_info)
test_trade_info = np.vstack(test_trade_info)

100%|██████████| 135/135 [00:00<00:00, 221.67it/s]


# 모델

In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.losses import mse, binary_crossentropy
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Concatenate, Conv1D, Reshape, Flatten, MaxPool1D, GlobalMaxPool1D, GlobalAvgPool1D, Multiply, Dot, Input, Attention, BatchNormalization, Dropout, Lambda, Input, Dense, Concatenate
from tensorflow.keras import backend as K
from tensorflow.keras import metrics
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping



# tf.config.experimental_run_functions_eagerly(True)

class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)


    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

    # def get_config(self):
    #     config = super().get_config().copy()
    #     config.update({
    #         'embed_dim': self.embed_dim,
    #         'num_heads': self.num_heads,
    #         'ff_dim': self.ff_dim
    #     })
    #     return config

class PositionEmbedding(layers.Layer):
    def __init__(self, maxlen, embed_dim):
        super(PositionEmbedding, self).__init__()
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        positions = tf.range(start=0, limit=self.maxlen, delta=1)
        positions = self.pos_emb(positions)
        return x + positions
    

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [19]:
import random as rn
import os
os.environ['PYTHONHASHSEED'] = '0'
seed_num = 77
np.random.seed(seed_num)
rn.seed(seed_num)
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
from tensorflow.keras import backend as K
tf.random.set_seed(seed_num)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)


# 200706 Transformer Model 2
num_layers = 2
num_heads = 8  # Number of attention heads
embed_dim = 64
#embed_dim = 
ff_dim = 4*embed_dim  # Hidden layer size in feed forward network inside transformer

num_features = 8
time_step = 20


trade_inputs = layers.Input(shape = (trade_info.shape[1],))
stock_inputs = layers.Input(shape = (time_step, num_features))

x = layers.Conv1D(filters = embed_dim, kernel_size=1, padding='same')(stock_inputs)
#x = layers.Reshape((1,embed_dim))(dense_total)
#embed_dim = num_features
pos_encoding_layer = PositionEmbedding(time_step, embed_dim)
x = pos_encoding_layer(x)
for _ in range(num_layers):
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(8, activation="relu")(x)
x2 = layers.Dense(128, activation='relu')(trade_inputs)
x = layers.Concatenate()([x, x2])
x = layers.Dropout(0.1)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1)(x)
model = Model([stock_inputs, trade_inputs], [outputs])
model.summary()




Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 20, 8)]      0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 20, 64)       576         input_2[0][0]                    
__________________________________________________________________________________________________
position_embedding (PositionEmb (None, 20, 64)       1280        conv1d[0][0]                     
__________________________________________________________________________________________________
transformer_block (TransformerB (None, 20, 64)       49984       position_embedding[0][0]         
_______________________________________________________________________________________

In [20]:
optimizer = keras.optimizers.Adam(lr=0.0005)
model.compile(optimizer=optimizer, loss=['mse'])

early_stop = EarlyStopping(monitor='val_loss', patience=10)
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')
model.fit(x=[stock_info, trade_info], y=y_list, batch_size=256, epochs=100, 
          validation_split=0.2,
          callbacks=[early_stop, reduce_lr_loss],
          shuffle=True)

Epoch 1/100
52/52 [==============================] - 4s 72ms/step - loss: 0.1515 - val_loss: 0.0168
Epoch 2/100
52/52 [==============================] - 1s 15ms/step - loss: 0.0523 - val_loss: 0.0179
Epoch 3/100
52/52 [==============================] - 1s 14ms/step - loss: 0.0411 - val_loss: 0.0163
Epoch 4/100
52/52 [==============================] - 1s 14ms/step - loss: 0.0365 - val_loss: 0.0183
Epoch 5/100
52/52 [==============================] - 1s 15ms/step - loss: 0.0339 - val_loss: 0.0162
Epoch 6/100
52/52 [==============================] - 1s 15ms/step - loss: 0.0314 - val_loss: 0.0174
Epoch 7/100
52/52 [==============================] - 1s 14ms/step - loss: 0.0302 - val_loss: 0.0158
Epoch 8/100
52/52 [==============================] - 1s 15ms/step - loss: 0.0279 - val_loss: 0.0142
Epoch 9/100
52/52 [==============================] - 1s 15ms/step - loss: 0.0264 - val_loss: 0.0130
Epoch 10/100
52/52 [==============================] - 1s 15ms/step - loss: 0.0250 - val_loss: 0.0120

In [21]:
y_pred = model.predict([test_stock_info[:,:,1:], test_trade_info])

In [22]:
df_pred = pd.DataFrame({'종목번호':np.array(test_stock_info)[:,0,0], '그룹번호':np.vstack(test_trade_info)[:,0], '예측':y_pred.flatten()})

In [23]:
df_pred['종목번호'] = le.inverse_transform(df_pred['종목번호'].astype(int))
df_pred['그룹번호'] = le2.inverse_transform(df_pred['그룹번호'].astype(int))

In [24]:
y_pred_list = []
for i in sorted(set(df_pred['그룹번호'])):
    y_pred_list.append(sorted(df_pred[df_pred['그룹번호']==i][['종목번호','예측']].sort_values(by='예측', ascending=False)['종목번호'][:3].values))

# LGBM

In [25]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from tsfresh import extract_features
from tqdm import tqdm

In [26]:
trade = pd.read_csv('trade_train.csv', index_col=0)
stock = pd.read_csv('stocks.csv', index_col=0)
answer = pd.read_csv('answer_sheet.csv')

In [27]:
trade = trade.merge(stock.loc[:, ['종목번호', '시장구분', '표준산업구분코드_대분류', '표준산업구분코드_중분류', '표준산업구분코드_소분류']].drop_duplicates(), on='종목번호')

In [28]:
trade['y'] = trade['매수고객수'] / trade['그룹내고객수']

In [29]:
gr2num = trade[['그룹번호', '그룹내고객수']].drop_duplicates().set_index('그룹번호').to_dict()['그룹내고객수']
trade_dict = trade.set_index(['기준년월', '그룹번호', '종목번호']).to_dict()
j_list = list(trade['기준년월'].unique()) + [202007]
k_list = trade['그룹번호'].unique()
for i in stock[stock['20년7월TOP3대상여부'] == 'Y']['종목번호'].unique():
    for j in j_list:
        for k in k_list:
            if (j, k, i) not in trade_dict['그룹내고객수']:
                trade_dict['y'][(j, k, i)] = 0
                trade_dict['그룹내고객수'][(j, k, i)] = gr2num[k]
                trade_dict['그룹내_매수여부'][(j, k, i)] = 'N'
                trade_dict['그룹내_매도여부'][(j, k, i)] = 'N'
                trade_dict['매수고객수'][(j, k, i)] = 0
                trade_dict['매도고객수'][(j, k, i)] = 0
                trade_dict['평균매수수량'][(j, k, i)] = -1
                trade_dict['평균매도수량'][(j, k, i)] = -1
                trade_dict['매수가격_중앙값'][(j, k, i)] = -1
                trade_dict['매도가격_중앙값'][(j, k, i)] = -1

In [30]:
trade = pd.DataFrame(trade_dict).reset_index()
trade = trade.rename({'level_0':'기준년월', 'level_1':'그룹번호', 'level_2':'종목번호'}, axis=1)
trade = trade.sort_values(['그룹번호', '기준년월', '종목번호'])

In [31]:
trade = trade[trade['종목번호'].isin(stock[stock['20년7월TOP3대상여부'] == 'Y']['종목번호'].unique())].reset_index(drop=True)

In [32]:
stock = stock[stock['20년7월TOP3대상여부'] == 'Y']

In [33]:
trade['전월_매수고객수'] = None
trade['평균_매수고객수'] = None
trade['전월_매도고객수'] = None
trade['전월_평균매수수량'] = None
trade['전월_평균매도수량'] = None
trade['전월_매수가격_중앙값'] = None
trade['전월_매도가격_중앙값'] = None
trade_dict = trade.set_index(['기준년월', '그룹번호', '종목번호']).to_dict()
trade_idx_dict = trade.to_dict()
buys = {}
for i in trade.index:
    pre_date = trade_idx_dict['기준년월'][i] - 1 if trade_idx_dict['기준년월'][i] != 202001 else 201912
    key = (pre_date, trade_idx_dict['그룹번호'][i], trade_idx_dict['종목번호'][i])
    if key in trade_dict['매수고객수']:
        trade_idx_dict['전월_매수고객수'][i] = trade_dict['매수고객수'][key]
        if key[1:] in buys:
            trade_idx_dict['평균_매수고객수'][i] = max(buys[key[1:]])
            buys[key[1:]].append(trade_dict['매수고객수'][key])
        else:
            buys[key[1:]] = [trade_dict['매수고객수'][key]]
        trade_idx_dict['전월_매도고객수'][i] = trade_dict['매도고객수'][key]
        trade_idx_dict['전월_평균매수수량'][i] = trade_dict['평균매수수량'][key]
        trade_idx_dict['전월_평균매도수량'][i] = trade_dict['평균매도수량'][key]
        trade_idx_dict['전월_매수가격_중앙값'][i] = trade_dict['매수가격_중앙값'][key]
        trade_idx_dict['전월_매도가격_중앙값'][i] = trade_dict['매도가격_중앙값'][key]

In [34]:
trade = pd.DataFrame(trade_idx_dict)

In [35]:
trade.fillna(0, inplace=True)

In [36]:
stock = stock.sort_values(['종목번호', '기준일자'])

In [37]:
pre_month = 201906
price = 0
stock['종가스케일'] = None
stock_dict = stock.to_dict()
for i in stock.index:
    now_month = stock_dict['기준일자'][i] // 100
    if now_month != pre_month:
        pre_month = now_month
        price = stock_dict['종목종가'][i]
    stock_dict['종가스케일'][i] = stock_dict['종목종가'][i] / price

In [38]:
stock = pd.DataFrame(stock_dict)

In [39]:
for i in stock[(stock['종목시가'] == 0)].index:
    stock.loc[i, '종목시가'] = stock.loc[i, '종목종가']
    stock.loc[i, '종목고가'] = stock.loc[i, '종목종가']
    stock.loc[i, '종목저가'] = stock.loc[i, '종목종가']

In [40]:
stock = stock.reset_index(drop=True)

In [41]:
stock['기준년월'] = stock['기준일자'].apply(lambda x : x // 100)

In [42]:
stock['7일변화량'] = None
stock['3일변화량'] = None
stock['연속상승'] = 0
stock['상승여부'] = 0
pre_month = 201906
stock_dict = stock.set_index(['종목번호', '기준일자']).to_dict()
stock_idx_dict = stock.to_dict()
for i in stock.index:
    if stock_idx_dict['기준년월'] != pre_month:
        stock_idx_dict['연속상승'][i] = 0
        stock_idx_dict['상승여부'][i] = 0
        pre_month = stock_idx_dict['기준년월']
    else:
        if stock_idx_dict['종목종가'][i] > stock_idx_dict['종목종가'][i-1]:
            stock_idx_dict['연속상승'][i] = stock_idx_dict['연속상승'][i-1] + 1
            stock_idx_dict['상승여부'][i] = 1
        
    if (stock_idx_dict['종목번호'][i], stock_idx_dict['기준일자'][i] - 7) in stock_dict['종목종가']:
        stock_idx_dict['7일변화량'][i] = stock_idx_dict['종목종가'][i] / stock_dict['종목종가'][(stock_idx_dict['종목번호'][i], stock_idx_dict['기준일자'][i] - 7)]
    if (stock_idx_dict['종목번호'][i], stock_idx_dict['기준일자'][i] - 3) in stock_dict['종목종가']:
        stock_idx_dict['3일변화량'][i] = stock_idx_dict['종목종가'][i] / stock_dict['종목종가'][(stock_idx_dict['종목번호'][i], stock_idx_dict['기준일자'][i] - 3)]

In [43]:
stock = pd.DataFrame(stock_idx_dict)

In [44]:
stock = stock[stock['종목번호'].isin(set(trade['종목번호']))]

In [45]:
stock = stock.sort_values(['종목번호', '기준일자'])

In [46]:
stock['id'] = stock['기준년월'].astype('str') + '_' + stock['종목번호']

In [47]:
stock['가격_변화량'] = (stock['종목종가'] - stock['종목시가']) / stock['종목시가']
stock['고가_변화량'] = (stock['종목고가'] - stock['종목시가']) / stock['종목시가']
stock['저가_변화량'] = (stock['종목저가'] - stock['종목시가']) / stock['종목시가']
stock['저가_고가_변화량'] = (stock['종목고가'] - stock['종목저가']) / stock['종목저가']

In [48]:
stock['가격_변화비율'] = 1 + stock['가격_변화량']
stock['고가_변화비율'] = 1 + stock['고가_변화량']
stock['저가_변화비율'] = 1 + stock['저가_변화량']
stock['저가_고가_변화비율'] = 1 + stock['저가_고가_변화량']

In [49]:
trade['id'] = trade['기준년월'].astype('str') + '_' + trade['종목번호']

In [50]:
for i in range(23):
    trade['가격{}'.format(i)] = None
    trade['전월가격{}'.format(i)] = None
    
trade_dict = trade.to_dict()
stock_dict = stock.set_index(['종목번호', '기준일자']).to_dict()

for i in tqdm(trade.index):
    j = 0
    for day in range(1, 32):
        day_tmp = trade_dict['기준년월'][i] * 100 + day
        try:
            trade_dict['가격{}'.format(j)][i] = stock_dict['종가스케일'][(trade_dict['종목번호'][i], day_tmp)]
            j += 1
        except:
            pass
    pre_month = trade.loc[i, '기준년월'] - 1 if trade.loc[i, '기준년월'] != 202001 else 201912
    j = 0
    for day in range(1, 32):
        day_tmp = pre_month * 100 + day
        try:
            trade_dict['전월가격{}'.format(j)][i] = stock_dict['종가스케일'][(trade_dict['종목번호'][i], day_tmp)]
            j += 1
        except:
            pass
    
trade = pd.DataFrame(trade_dict)

100%|██████████| 84240/84240 [00:10<00:00, 7890.51it/s]


In [51]:
timeseries = stock[stock['id'].isin(set(trade['id']))][['id', '기준일자', '상승여부']]

In [52]:
timeseries = timeseries.fillna(1)

In [53]:
features_filtered_direct = extract_features(timeseries, column_id='id', column_sort='기준일자')

Feature Extraction: 100%|██████████| 115/115 [00:05<00:00, 19.86it/s]


In [54]:
features_filtered_direct = features_filtered_direct.reset_index()
features_filtered_direct = features_filtered_direct.rename({'index':'id'}, axis=1)

In [55]:
trade = trade.merge(features_filtered_direct, on='id')

In [56]:
for c in trade.columns:
    if not (trade[c].dtype == 'float64' or trade[c].dtype == 'int'):
        trade[c] = trade[c].astype('category')

In [57]:
trade['pre_y'] = trade['전월_매수고객수'] / trade['그룹내고객수']

In [58]:
for c in trade.columns:
    if not (trade[c].dtype == 'float64' or trade[c].dtype == 'int'):
        trade[c] = trade[c].astype('category')

In [59]:
trade['예측'] = None

price = ['가격{}'.format(i) for i in range(18)]
pre_price = ['전월가격{}'.format(i) for i in range(18)]
X = trade[['pre_y', '상승여부__sum_values'] + price + pre_price]
X['가격0'] = X['가격0'].astype('float')
X = X.dropna()


X_ = X[(trade['기준년월'] > 201907) & (trade['기준년월'] != 202007)]
X__ = X[(trade['기준년월']== 202007)]
lg = LGBMRegressor(num_leaves=7, reg_lambda=0.1)
lg.fit(X_, trade.loc[X_.index, 'y'])
trade['예측'][(trade['기준년월']== 202007)] = lg.predict(X__)

<ipython-input-59-c8da248f7752>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['가격0'] = X['가격0'].astype('float')
<ipython-input-59-c8da248f7752>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_ = X[(trade['기준년월'] > 201907) & (trade['기준년월'] != 202007)]
<ipython-input-59-c8da248f7752>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X__ = X[(trade['기준년월']== 202007)]
<ipython-input-59-c8da248f7752>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trade['예측'][(trade['기준년월']== 202007)] = lg.pred

In [60]:
for i in answer.index:
    tmp = trade[['종목번호', '예측']][(trade['기준년월'] == 202007) & (trade['그룹번호'] == answer.loc[i, '그룹명'])].sort_values('예측', ascending=False)
    tmp2 = sorted([tmp.iloc[0, 0], tmp.iloc[1, 0], tmp.iloc[2, 0]])
    answer.loc[i, '종목번호1'] = tmp2[0]
    answer.loc[i, '종목번호2'] = tmp2[1]
    answer.loc[i, '종목번호3'] = tmp2[2]

In [61]:
json_df = trade[['그룹번호', '종목번호', '예측']][(trade['기준년월']== 202007)].sort_values(['그룹번호', '예측', '종목번호'], ascending=False)

# ensemble

In [62]:
df1 = json_df
df2 = df_pred

In [63]:
df = pd.merge(df1, df2, how='inner', on=['그룹번호','종목번호'])

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['예측_x'] = scaler.fit_transform(df['예측_x'].values.reshape(-1,1))
df['예측_y'] = scaler.fit_transform(df['예측_y'].values.reshape(-1,1))

w = 0.65
df['예측'] = w*df['예측_x'] + (1-w)*df['예측_y']

In [64]:
y_pred_list = []
for i in sorted(set(df['그룹번호'])):
    y_pred_list.append(sorted(df[df['그룹번호']==i][['종목번호','예측']].sort_values(by='예측', ascending=False)['종목번호'][:3].values))

In [65]:
ans = pd.read_csv("answer_sheet.csv")
ans.iloc[:,1:] = y_pred_list
ans.to_csv('submit.csv', index=False)